In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [7]:
df = pd.read_csv('insurance.csv')
df.head(2)


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523


In [8]:
df['region'].value_counts()

region
southeast    364
southwest    325
northwest    325
northeast    324
Name: count, dtype: int64

In [9]:
# Feature Engg

label_sex = LabelEncoder()
label_smoke = LabelEncoder()
df['sex'] = label_sex.fit_transform(df['sex'])
df['smoker'] = label_smoke.fit_transform(df['smoker'])

onehot_region = OneHotEncoder(sparse_output=False)
onehot_region.fit(df[['region']])
col_names = onehot_region.get_feature_names_out(['region'])
region_encoder = onehot_region.transform(df[['region']])
encoded_df = pd.DataFrame(region_encoder,columns = col_names )


In [11]:
df = pd.concat([df.drop('region',axis = 1),encoded_df],axis= 1)
df

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0.0,0.0,0.0,1.0
1,18,1,33.770,1,0,1725.55230,0.0,0.0,1.0,0.0
2,28,1,33.000,3,0,4449.46200,0.0,0.0,1.0,0.0
3,33,1,22.705,0,0,21984.47061,0.0,1.0,0.0,0.0
4,32,1,28.880,0,0,3866.85520,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,10600.54830,0.0,1.0,0.0,0.0
1334,18,0,31.920,0,0,2205.98080,1.0,0.0,0.0,0.0
1335,18,0,36.850,0,0,1629.83350,0.0,0.0,1.0,0.0
1336,21,0,25.800,0,0,2007.94500,0.0,0.0,0.0,1.0


In [ ]:
# Saving the encoders in Pickle file
with op